## Cuaderno para ejecutar telegram-tracker

### Montamos nuestro drive en Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Instalamos telethon

In [ ]:
!python3 -m pip install --upgrade telethon

### Ejecutamos el script principal de telegram-tracket



```
usage: main.py
       [-h]
       --telegram-channel
       TELEGRAM_CHANNEL
       --batch-file
       BATCH_FILE
       [--limit-download-to-channel-metadata]
       [--output OUTPUT]
       [--min-id MIN_ID]

Opciones:
 --telegram-channel  para descargar un canal
 --batch-file  Para descargar una lista de canales
 --limit-download-to-channel-metadata bajar solo los metadatos del canal (sin mensajes)
 --output directorio donde descargar los datos
 --min-id MIN_ID descargar sólo datos recientes (mayores del id especificado)
```



### Extraer un canal

Será la raiz de la búsqueda


In [ ]:
import os
# Establecer el directorio de trabajo
os.chdir("/content/drive/MyDrive/telegram-tracker-t-hoarder_tg-main")
# Poner nombre del canal raiz
%env root = rubnpulido
#descargar canal
!python main.py --telegram-channel ${root} --output ./data/${root}
# Convertir json en csv y excel
!python build-datasets.py --data-path  ./data/${root}

### Extraer canales relacionados con el canal raíz


In [ ]:
!cut -d "," -f 2-2  ./data/${root}/counter.csv |  tail -n +2 | awk '$1!="HiddenSender"'> ./data/${root}/${root}_n2.csv
!python main.py --batch-file ./data/${root}/${root}_n2.csv --output ./data/${root}_n2
# convertir json en csv y excel
!python build-datasets.py --data-path  ./data/${root}_n2

### Extraer varios canales

In [ ]:
import os
import csv
# Establecer el directorio de trabajo
os.chdir("/content/drive/MyDrive/telegram-tracker-t-hoarder_tg-main")
# Poner nombre conjunto de canles
%env group = xxxxxx
!mkdir ./data/${group}/ 2> /dev/nul
channels = input('Enter a comma separated list of channels:    ')
channels = channels.split (",")
with open('list_channels.txt', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter="\n")
    writer.writerow(channels)
!mv list_channels.txt ./data/${group}/
#descargar canales
!python main.py --batch-file ./data/${group}/list_channels.txt --output ./data/${group}
# Convertir json en csv y excel
!python build-datasets.py --data-path  ./data/${group}